## Ciabattini

- bulk fermentation: 3hrs, fold twice
- divide & shape
- final fermentation: 1.5 hrs


In [68]:
import pandas as pd
grams_per_ounce = 28.3495

In [69]:
num_loaves = 45
num_pounds = 0
num_ounces = 2.5
num_grams = 0

In [70]:
loaf_weight = (num_pounds * 16 + num_ounces) * grams_per_ounce + num_grams
total_weight = (num_loaves * loaf_weight)
print ("{:,.0f} loaves at {:,.0f} grams".
       format(num_loaves, loaf_weight, num_pounds, num_ounces))
print ("total weight: {:,.2f} grams".format(total_weight,total_weight * grams_per_ounce))

45 loaves at 71 grams
total weight: 3,189.32 grams


In [71]:
formula = pd.DataFrame.from_dict({
    "ap-flour": 100, 
    "water": 73, 
    "yeast": 1.1,
    "salt": 2,
}, orient='index', columns=["baker%"])
print ("overall formula")
formula_total = formula['baker%'].sum()
print("overall formula total = %s%%" % formula_total)
formula

overall formula
overall formula total = 176.1%


,baker%
ap-flour,100.0
water,73.0
yeast,1.1
salt,2.0


In [72]:
pre_ferment = .30
poolish = pd.DataFrame.from_dict({
    "ap-flour": 100,
    "water": 100, 
    "yeast": 0.2
}, orient='index', columns=["baker%"])
print ("poolish")
poolish_total = poolish['baker%'].sum()
print("poolish total = %s%%" % poolish_total)
poolish

poolish
poolish total = 200.2%


,baker%
ap-flour,100.0
water,100.0
yeast,0.2


In [73]:
poolish_weight = total_weight * 100 / formula_total * pre_ferment * poolish_total/poolish.at['ap-flour','baker%']
print ("poolish weight = {:,.0f} grams".format(poolish_weight))
poolish['grams'] = poolish.apply(lambda row: '{:.0f}'.format(row['baker%']*poolish_weight/poolish_total), axis=1)
poolish['oz'] = poolish.apply(lambda row: '{:.1f}'.format(float(row['grams']) / grams_per_ounce), axis=1)
poolish = poolish[['baker%', 'grams', 'oz']]
poolish

poolish weight = 1,088 grams


,baker%,grams,oz
ap-flour,100.0,543,19.2
water,100.0,543,19.2
yeast,0.2,1,0.0


In [74]:
final_dough = formula.copy()
final_dough['grams'] = final_dough.apply(lambda row: '{:.0f}'.format(row['baker%']*total_weight / formula_total), axis=1)
final_dough.at['poolish','grams'] = '{:.0f}'.format(poolish_weight)
# Convert baker% column to object type to allow mixed data types
final_dough['baker%'] = final_dough['baker%'].astype('object')
final_dough.at['poolish','baker%'] = '{:.0f}'.format(poolish_weight / float(final_dough.at['ap-flour','grams']) * 100)
for index, row in poolish.iterrows():
    final_dough.at[index,'grams'] = float(final_dough.at[index,'grams']) - float(row['grams'])
final_dough['oz'] = final_dough.apply(lambda row: '{:,.1f}'.format(float(row['grams'])/grams_per_ounce), axis=1)
final_dough = final_dough[['baker%', 'grams', 'oz']]
print("final dough")
final_dough

final dough


,baker%,grams,oz
ap-flour,100.0,1268.0,44.7
water,73.0,779.0,27.5
yeast,1.1,19.0,0.7
salt,2.0,36,1.3
poolish,60,1088,38.4
